# 1 Imports

In [2]:
import os
import pandas as pd
import sqlalchemy as sql # 用來和 db 連動的 library，可以和 SQLite, Postgres, MySQL, Microsoft SQL Server, and more 互動

# 2 DB 基本操作

## 2.1 [增] 建立 database

* 我們打算用 SQLite 這種 database，那他實際上就會放在我們專案中的某個資料夾裡面
* 所以，我先建立一個資料夾給他，之後 db 的東西就會放在裡面：

In [4]:
# os.mkdir("00_database")

* 接著，連接和建立某個database，作法都一樣：  
  * 先 create 一個 engine (就是你要連到哪個資料庫/url)
  * 然後用 `.connect()` 做連接  
* 那如果這個資料庫之前存在，就會幫你連上。如果不存在，就會幫你 create  
* 所以，現在練習一下，我想建立 `bike_orders_database.sqlite` 這個資料庫，並把他放在 `00_database` 這個資料夾裡面：

In [17]:
engine = sql.create_engine("sqlite:///00_database/bike_orders_database.sqlite") # sqlite: 後面的前兩個斜線，就跟 http:// 意思一樣，那第三個斜線，是因為我要指到本地資料夾，所以才會這樣寫 /00_database/...
conn = engine.connect()

## 2.2 [增] 建立 table

* 接下來，我們要讀取原本的 excel file，並把他寫到這個資料庫裡面

In [7]:
bikes_df = pd.read_excel("./00_data_raw/bikes.xlsx", engine='openpyxl')
bikeshops_df = pd.read_excel("./00_data_raw/bikeshops.xlsx", engine='openpyxl')
orderlines_df = pd.read_excel("./00_data_raw/orderlines2.xlsx", engine='openpyxl')

* 寫入時，pandas 對 DataFrame 這種物件，本身就有 method 來幫你寫資料，寫成 `df.to_sql("db_table_name", con = conn, 其他參數)`。可用 `?pd.DataFrame.to_sql` 來查詢用法
* 這就像 R 的 `dbWriteTable(con = conn, df, "db_table_name", 其他參數)` 一樣。
* 常見的其他參數，例如 `if_exists = "replace"` 的意思，就是如果這張 table 已經在 db 裡面，那我就取代掉他。所以這個參數就像 R 的 `overwrite = TRUE`  

In [10]:
bikes_df.to_sql("bikes", con = conn, if_exists="replace")
bikeshops_df.to_sql("bikeshops", con = conn, if_exists="replace")
orderlines_df.to_sql("orderlines", con = conn, if_exists="replace")

## 2.3 [查] 讀 database 裡有哪些 table

In [19]:
inspector = sql.inspect(conn) # 做出來的 inpsector 是一個物件，所以他也有自己的 methods
inspector.get_table_names() # 讀現在的 db 裡面有哪些 tables  

['bikes', 'bikeshops', 'orderlines']

* 我們也可以看看，現在這個 database 的 schema 叫什麼(這個 db 的 architecture)

In [20]:
inspector.get_schema_names()

['main']

* 預設的 schema 的 名稱就是 `main`，

## 2.4 [查] 讀 table

* 讀檔時，是用 `pd.read_sql("sql語句", con = 連線物件)` 來處理
* 這就像 R 的 `dbGetQuery(con = 連線物件, "sql語句")`

In [13]:
pd.read_sql("select * from bikes", con = conn).head()

,index,bike.id,model,description,price
0,0,1,Supersix Evo Black Inc.,Road - Elite Road - Carbon,12790
1,1,2,Supersix Evo Hi-Mod Team,Road - Elite Road - Carbon,10660
2,2,3,Supersix Evo Hi-Mod Dura Ace 1,Road - Elite Road - Carbon,7990
3,3,4,Supersix Evo Hi-Mod Dura Ace 2,Road - Elite Road - Carbon,5330
4,4,5,Supersix Evo Hi-Mod Utegra,Road - Elite Road - Carbon,4260


* 那這邊順便提一招 python 的 string format，就是我要把變數名稱塞到字串裡的做法 (R 是直接用 `paste("某字串", 某變數)` 的方式處理)
* python 會這樣寫 `f"某字串 {某變數}"` ，其中的 `f` 就是 format 的意思。
* 來看個例子：

In [21]:
aa = "Hank"
f"My name is {aa}"

'My name is Hank'

* 那學會這招後，我們就可以用在資料讀取上，例如：

In [23]:
table = inspector.get_table_names() # 所以現在的 table 的內容是 ['bikes', 'bikeshops', 'orderlines']
pd.read_sql(f"select * from {table[1]}", con = conn).head() # 我去讀 bikeshops 這個 table

,index,bikeshop.id,bikeshop.name,location
0,0,1,Pittsburgh Mountain Machines,"Pittsburgh, PA"
1,1,2,Ithaca Mountain Climbers,"Ithaca, NY"
2,2,3,Columbus Race Equipment,"Columbus, OH"
3,3,4,Detroit Cycles,"Detroit, MI"
4,4,5,Cincinnati Speed,"Cincinnati, OH"


* 這招等等會常用，因為我們會想用 for 回圈，把 database 裡的 table 都讀一遍。

## 2.4 斷開連結

* 有禮貌的小孩，用完就要把連結斷開  
* 使用語法就是 `conn.close()` 就好

In [15]:
conn.close()

* 這時，如果你在做一次 `pd.read_sql("select * from bikes", con = conn)` ，就會得到 error: `StatementError: (sqlalchemy.exc.ResourceClosedError) This Connection is closed`  

# 3 GUI 工具

* 就像在公司用 oracle database 時，有一個 oracle 的 GUI 可以讓我們看 database 裡面有什麼東西，並直接做操作一樣。我們可以把 VSCode 當我們的 GUI，只要下載對應的 extension 即可
* 例如，我們下載 `SQLite` 這個 extension，安裝完成後，我就在 VSCode 中，按 `cmd + shift + p` ，並輸入 `sqlite: Open Database`，然後指到我們存檔的那個 sqlite db，就會在左邊的 pannel，看到 `SQLITE EXPLORER`
* 點開 SQLITE EXPLORER，就可以點裡面的 table，看一下長怎麼樣了。
* 要關閉連結時，一樣用 `cmd + shift + p` ，然後輸入 `sqlite: Close Database`，就可以關閉連結。